# kaggle 우승작으로 배우는 머신러닝_1 

코드 실습

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
print(os.listdir("../input"))

# 간단한 설명

고객의 다양한 재정적 의사결정을 돕기 위해 '맞춤형 추천' 제품을 제공하려는 은행. 고객의 과거 이력과 유사한 고객군의 데이터를 기반으로 '다음달에 해당 고객이 무슨 제품을 사용할지' 예측하는 것.

자사의 금융제품을 사용하는 고객에게, 다른 금융제품을 소개해 만족도를 높이고 매출을 올리기 위한 것.

# 평가 척도

'신규 구매 제품': 지난달까지는 보유하지 않은 금융제품 중 이번달에 신규로 구매하게 되는 제품을 말한다. 이미 보유한 제품을 지속적으로 사용하는 것은 제외, 지난달 보유 제품을 이번달에 해지하는 것도 제외한다.

"16-05-28(지난 달) 시점에 보유하지 않은 금융 제품 중 18-06-28(이번 달)에 구매할 것으로 예측되는 제품 상위 7개"의 제출.

### 평가 척도는 Mean Average Percision @ 7

예컨대 예측 결과가 [1,0,0,1,1,1,0] 이라면, 첫 번째의 예측 정확도는 1/1, 네 번째 예측 정확도는 2/4, 그 이후로 3/5, 4/6이다.

정확도의 합을 정답의 갯수만큼 나눈 숫자 = (1/1+2/4+3/5+4/6) / 4 = .69

모든 예측 결과물의 average precision의 평균값을 말한다.

예측 순서에 상당히 예민한 평가 척도. 정답을 앞쪽에 예측할수록 더 좋은 점수를 받을 수 있다는 뜻이다. 예컨대 7개 중 앞 4개를 제대로 예측했다면 Average precision은 1이 된다.

반대로 4개 정답이 뒤에서부터 4개일 경우 (1/4+2/5+3/6+4/8) / 4 = .43


# 주요 접근

Tabular 형태의 시계열 데이터가 제공된다. 'A라는 시간에 B라는 고객이 C라는 제품을 구매했다'는 정보가 담긴 데이터이며, B에 관련된 다양한 정보가 포함된다. 예컨대 나이, 직업유무, 결혼유무, 고객 등급, 거주지, 국적 등등. 또한 C에 관련된 제품 분류, 가격, 인기도, 특징 등등.

모델이 예측해야 하는 target value 식별에 도움 되는 정보가 많을수록 좋은 데이터다.

시계열 데이터를 다루는 경진대회는 '유의미한 파생변수 생성'을 위한 기술이 있다. 
1. 날짜/시간 정보가 포함된 데이터의 경우 '주중/주말 여부', '공휴일 여부', '아침/낮/밤', '계절구분', '학기/시험기간/방학'등 다양한 파생변수를 생성할 수 있다. 
2. 과거 데이터를 활용하는 lag 데이터를 파생 변수로도 사용할 수 있음. 몇 달 전에 고객이 구매한 제품 데이터를 이번 달 예측에 활용하는 식이다.

식별력이 있는 파생변수는 모델마다, 문제마다 다르므로 시도해봐야 한다.

### Tabular 형태의 시계열 데이터를 다루는 경진대회는 딥러닝 모델보다 트리 기반의 앙상블 모델이 더 효과가 좋다.

scikit learn에서도 다양한 트리모델을 지원하지만, 케글 경진대회에서 많이 쓰인 xgboost와 lightgbm을 사용할 예정

특히 xgboost는 가장 좋은 성능과 빠른 속도 덕분에 많이 쓰이는 모델. boosting tree는 간단히 요약하자면 
* '하나의 트리 모델을 학습시킨 후, 해당 트리모델의 성능이 낮은 부분을 보완하는 다른 트리모델을 학습시키는 방식으로 수많은 트리 모델을 순차적으로 학습시키며 성능을 개선하는 모델'



In [ ]:
trn = pd.read_csv('../input/train_ver2.csv')
trn.head()

In [ ]:
trn.shape
# 데이터 크기는 약 1,300만 개. 고객마다 48개의 변수 존재

In [ ]:
for col in trn.columns:
    print(trn[col].head(),trn[col].dtype)

fecha_dato : datetime 변수가 아니라 object 형식
age : 마찬가지로 정수 변수가 아니라 object. 변형 필요
renta: 가구 총수입을 의미하는 변수. 결측치가 존재. 변환 필요

## EDA 첫 단계에서 모든 걸 이해하려고 접근할 필요 없다

단계적으로 데이터에 익숙해지는 과정 필요. 간단한 시각화 정도?
- 경진대회 데이터가 어떻게 생겼는지
- 어떤 변수들이 존재하는지
- data type 확인하고 전처리 수행할 변수 찾기

정도 느낌으로도 충분하다

In [ ]:
trn.info()

1~24까지는 고객 관련 변수, 그 다음 변수들은 '금융 제품 변수'

머신러닝 학습을 위해서는 모든 변수가 int / float로 바뀌어야 한다. 그  외의 타입은 인수로 받지 못함. 따라서 '어떤 변수'를 변환해야 하는지 확인할 수 있다.

fecha_dato : 날짜

ncodpers : 고객 고유식별번호

ind_empleado : 고용 지표. a = active, b = ex employed, f = filial, n = not employeed, p = passive

pais_residencia: 거주지

fecha_alta : 고객이 은행과 첫 계약체결한 날짜

ind_nuevo: 신규 고객지표( 6개월 이내 신규고객이면 1)

antiguedad : 거래 누적기간(월)

indrel: 고객 등급( 1: 1등급, 99: 해당 달에 고객 1등급이 해제되는 1등급 고객)

ult_fec_cli_1t : 1등급 고객으로의 마지막 날짜

indrel_1mes: 월초 기준 고객등급 (1: 1등급, 2: co-owner, P: potential, 3: former primary, 4: former co-owenr)

tiprel_1mes: 월초 기준 고객관계유형 (a: active, i: inactive, p: former customer, r: potential)

indresi: 거주지표 (고객의 거주국가와 은행 위치국가 동일여부. s: yes, n: no)

indext: 외국인 지표 (태어난 국가와 은행이 위치한 국가 동일 여부)

conyuemp: 배우자 지표. (1:은행 직원을 배우자로 둔 고객)

canal_entrada: 고객 유입채널

indfall : 고객 사망여부

tipodom : 주소유형(1: primary address)

cod_prov: 지방 코드 (주소 기반)

nomprov : 지방 이름

ind_actividad_cliente : 활발성 지표 (1: active, 2: inactive)

renta : 가구 총수입

segmento: 분류 (1 : vip, 2: 개인, 3: 대졸)


## 수치형 / 범주형 변수 확인

24개의 고객 관련 변수 확인하기,

In [ ]:
num_col = [col for col in trn.columns[:24] if trn[col].dtype in ['int64','float64']]
trn[num_col].describe()
# 수치형 변수는 ncodpers, ind_nuevo, indrel, tipodom, cod_prov, ind_actividad_cliente, renta


In [ ]:
cat_col = [col for col in trn.columns[:24] if trn[col].dtype in ['O']]
trn[cat_col].describe()
# count : shape에 출력된 값보다 작다면 결측치가 있다는 뜻.
# unique: 고유값이 몇 개 있는지
# top : 가장 빈출 빈도 높은 데이터
# freq : 최빈 데이터의 빈도수. count 대비 최빈값이 얼마인지를 확인하면, 대략적인 분포를 가늠할 수 있다. 예컨대 ind_empleado는 N이 전체의 99% 차지함. (편중 정도)

In [ ]:
# 고유값 직접 출력해보기
for col in cat_col:
    uniq = np.unique(trn[col].astype(str))
    print('-'*50)
    print("# col {}, n uniq {}, uniq {}".format(col, len(uniq), uniq))


이렇게 출력되는 데이터 타입을 보면서, 눈에 띄는 특징은 노트에 별도로 기록하는 것을 권장한다. '변수 아이디어'로 해당 변수를 유의미하게 활용할 수 있는 feature engineering idea를 기록하는 것도 좋다.

자기만의 아이디어를 코드로 구현하고 실험으로 검증하는 것이 머신러닝 학습의 지름길이다.

## 시각화로 데이터 살펴보기

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
# 각 변수의 histogram 그리기. 고객 고유식별번호와 총수입인 ncodpers, renta는 고유값이 너무 많아 시각화에 시간이 오래 걸리므로 제외
skip_col= ['ncodpers','renta']

for idx, col in enumerate(trn.columns):
    if col in skip_col:
        continue
    
    print('-' * 50)
    print('col : ', col)
    
    # 그래프 크기 설정
    f, ax = plt.subplots(figsize=(20,15))
    # seaborn 사용한 막대그래프 생성 
    sns.countplot(x = col, data = trn, alpha = .5)
    plt.show()

fecha_dato : 첫 6개월은 고객 데이터 수가 같으며, 7월 28일 이후부터 매달 고객이 증가한다.

indrel_1mes : 1, 1.0, 1.0이 분리되어 나타나 있다. 데이터 타입 통일이 필요함

age, antiguedad: 수치형 변수인 나이가 중간에 끊긴 모습. 아마 object 형태로 저장되어 있어서인 것으로 보인다. 

fecha_alta : 95년부터 16년까지 폭넓은 값의 데이터. 95년 부근 고객과 16년 부근이 높은 빈도인 것으로 보아 장기고객 & 신규고객 비율이 둘 다 높을 것.

**시각화해서 보이는 정보도 노트에 꼼꼼히 적어두는 것을 추천한다.**



In [ ]:
# 시계열성을 포함한 시각화 기법으로 24개의 제품 변수를 자세히 들여다보기.
# 각 제품은 고객이 보유하면 1, 보유하지 않을 경우 0이다.
# 각 제품 변수를 다 더한 값은 제품 보유 수준을 의미한다.

# 월별 제품변수의 합을 누적 막대그래프 형태로 시각화하기. 누적 그래프를 쓰는 이유는 '서로 다른 제품의 차이를 함께 시각화하기 위해'

# 날짜 데이터를 기준으로 분석하기 위해 날짜 데이터 추출
months = trn['fecha_dato'].unique().tolist()
label_cols = trn.columns[24:].tolist()

label_over_time=[]

for i in range(len(label_cols)):
    # 매월, 각 제품의 총합을 groupby.agg(sum) 으로 계산 후 label_sum에 저장한다.
    label_sum = trn.groupby(['fecha_dato'])[label_cols[i]].agg(sum)
    label_over_time.append(label_sum.tolist())
    
label_sum_over_time =[]
for i in range(len(label_cols)):
    # 누적 막대그래프의 시각화를 위해 n번째 제품의 총합을 1~n번째 제품의 총합으로 바꾼다
    label_sum_over_time.append(np.asarray(label_over_time[i:]).sum(axis=0))
    
color_list = ['#F5B7B1','#D2B4DE','#AED6F1','#A2D9CE','#ABEBC6','#F9E79F','#F5CBA7','#CCD1D1']
# 그림 크기를 정의한다
f, ax = plt.subplots(figsize=(30,15))
for i in range(len(label_cols)):
    # 24개 제품 historgram
    # x축은 월 데이터, y는 누적 총합.
    sns.barplot(x = months, y = label_sum_over_time[i],color = color_list[i%8], alpha=.7)
    
    
plt.legend([plt.Rectangle((0,0),1,1,fc=color_list[i%8], edgecolor = 'none') for i in range(len(label_cols))], label_cols, loc=1, ncol = 2, prop={'size':16})

상품의 총합은 증가하고 있다. (고객의 숫자가 꾸준히 늘어나고 있기 때문인 것으로 보인다고 분석)

절대값 대신 상대값으로 시각화를 시도한다면

In [ ]:
# label_sum_over_time의 값을 퍼센트 단위로 변환하기. 월마다 최댓값으로 나누고 100을 곱한다.
label_sum_percent = (label_sum_over_time / (1.*np.asarray(label_sum_over_time).max(axis=0))) * 100

# 앞선 코드와 동일한, 시각화 실행 코드
f, ax = plt.subplots(figsize=(30, 15))
for i in range(len(label_cols)):
    sns.barplot(x=months, y=label_sum_percent[i], color = color_list[i%8], alpha=0.7)
    
plt.legend([plt.Rectangle((0,0),1,1,fc=color_list[i%8], edgecolor = 'none') for i in range(len(label_cols))], \
           label_cols, loc=1, ncol = 2, prop={'size':16})

딱히 눈에 띄는 변화패턴이 있지는 않다.

## 데이터 분석의 목적을 잃지 않기.

컴퓨터 스크린에 뿌려진 데이터를 분석하다 보면 분석의 진짜 목적과 방향성을 잃어버리기 쉽다. 아무 생각 없이, 무의미한 분석방법을 실행하고자 어렵고 복잡한 코딩에 시간낭비하는 경우가 허다함.

아직 머신러닝 알고리즘 학습도 안 한 상태에서 '무엇을 시각화할지, 무엇을 분석할지' 정답을 찾기는 어렵다.

필자의 경험상, 예측을 위해 제공되는 변수(고객 변수)보다는 실제로 예측해야 하는 변수(금융 제품 변수)를 올바르게 분석해 이해도를 높이는 게 머신러닝 모델 구축에 도움이 됐다.

EDA를 통해 얻고자 하는 것
1. 데이터 기초 통계 & 시각화로 데이터 직접 들여다보기
2. 변수 아이디어 찾아내기
3. 예측 변수 24개의 특징 찾아내기


### 경진대회에서 예측해야 하는 건 '고객이 신규로 구매할 제품'이다. 보유 여부가 0에서 1로 전환되는 경우가 중요하지, 얼마나 오래 보유하느냐 등등은 중요하지 않다

앞선 누적 막대그래프는 '제품의 총 보유량'일 뿐, '신규 구매'의 월별 추이는 나타나지 않는다.

하지만 데이터에는 '신규 구매'라는 정보가 주어지지 않았으므로, '신규 구매'정보를 별도로 추출하는 과정이 필요하다.


In [ ]:
gc.collect()

In [ ]:
# 제품 변수를 prods에 list형태로 저장한다
prods = trn.columns[24:].tolist()

# 날짜를 숫자로 변환하는 함수이다. 2015-01-28은 1, 2016-06-28은 18로 변환된다
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date


# 날짜를 숫자로 변환하여 int_date에 저장한다
trn['int_date'] = trn['fecha_dato'].map(date_to_int).astype(np.int8)
#trn['int_date+1'] = trn['int_date']+1
# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
# trn_lag = trn.copy()
# trn_lag['int_date'] += 1
# trn_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in trn.columns]

In [ ]:
# # 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. Lag 데이터의 int_date는 1 밀려있기 때문에, 저번달의 제품 정보가 삽입된다.
# # 메모리 초과로 여기서부터 죽는다
#df_trn = trn.merge(trn_lag, on=['ncodpers','int_date'], how='left')
# # 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다
# del trn, trn_lag

In [ ]:


# # 저번달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
# for prod in prods:
#     prev = prod + '_prev'
#     df_trn[prev].fillna(0, inplace=True)

# # 원본 데이터에서의 제품 보유 여부 – lag데이터에서의 제품 보유 여부를 비교하여 신규 구매 변수 padd를 구한다
# for prod in prods:
#     padd = prod + '_add'
#     prev = prod + '_prev'
#     df_trn[padd] = ((df_trn[prod] == 1) & (df_trn[prev] == 0)).astype(np.int8)

# # 신규 구매 변수만을 추출하여 labels에 저장한다.
# add_cols = [prod + '_add' for prod in prods]
# labels = df_trn[add_cols].copy()
# labels.columns = prods
# #labels.to_csv('../input/labels.csv', index=False)

In [ ]:
# # 코드 1-12. 신규 구매 누적 막대 그래프를 시각화하기

# #labels = pd.read_csv('../input/labels.csv').astype(int)
# fecha_dato = trn['fecha_dato']

# labels['date'] = fecha_dato.fecha_dato
# months = np.unique(fecha_dato.fecha_dato).tolist()
# label_cols = labels.columns.tolist()[:24]

# label_over_time = []
# for i in range(len(label_cols)):
#     label_over_time.append(labels.groupby(['date'])[label_cols[i]].agg('sum').tolist())
    
# label_sum_over_time = []
# for i in range(len(label_cols)):
#     label_sum_over_time.append(np.asarray(label_over_time[i:]).sum(axis=0))
    
# color_list = ['#F5B7B1','#D2B4DE','#AED6F1','#A2D9CE','#ABEBC6','#F9E79F','#F5CBA7','#CCD1D1']

# f, ax = plt.subplots(figsize=(30, 15))
# for i in range(len(label_cols)):
#     sns.barplot(x=months, y=label_sum_over_time[i], color = color_list[i%8], alpha=0.7)
    
# plt.legend([plt.Rectangle((0,0),1,1,fc=color_list[i%8], edgecolor = 'none') for i in range(len(label_cols))], label_cols, loc=1, ncol = 2, prop={'size':16})

첫 달에 신규 구매 숫자가 압도적으로 많은 이유는, 데이터 첫 달에는 모든 보유제품이 신규 구매로 인식되기 때문이다. 다른 달의 신규 구매횟수를 자세히 보려면, 상대값 기준으로 다시 시각화한다.

In [ ]:
# # 코드 1-13. 신규 구매 누적 막대 그래프를 상대값으로 시각화하기

# label_sum_percent = (label_sum_over_time / (1.*np.asarray(label_sum_over_time).max(axis=0))) * 100

# f, ax = plt.subplots(figsize=(30, 15))
# for i in range(len(label_cols)):
#     sns.barplot(x=months, y=label_sum_percent[i], color = color_list[i%8], alpha=0.7)
    
# plt.legend([plt.Rectangle((0,0),1,1,fc=color_list[i%8], edgecolor = 'none') for i in range(len(label_cols))], \
#            label_cols, loc=1, ncol = 2, prop={'size':16})

1. ind_cco_fin_ult1 (당좌 예금)은 8월 여름에 가장 높은 값, 겨울에는 축소되는 계절 추이 (누적그래프 맨 위)
2. ind_deco_fin_ult1 (단기 예금)은 150628 시기에 특이하게 높은 값, 나머지는 값이 매우 낮다 (위에서 다섯 번째)
3. 급여, 연금 (ind_nomina_ult1, ind_nom_pens_ult1)은 당좌예금과 반대로 8월 여름에 가장 낮으며 160228 겨울에 가장 높은 값
4. 신규 구매빈도가 가장 높은 상위 5개 금융제품은 당좌예금, 신용카드, 급여, 연금, 직불카드이다 (ind_cco_fin,ult1, ind_tjcr_fin_ult1, ind_nomina_ult1, ind_nom_pens_ult1, ind_recibo_fin_ult1)

즉 데이터가 seasonality를 보인다. 이 경우 훈련 데이터를 몇 월로 지정하는가에 따라 머신러닝 모델 결과물이 많이 달라질 수 있다는 걸 의미함. 계절 변동성을 모델링하려면 '하나의 일반적인 모델을 구축할 건지, 계절에 따라 다수 모델을 구축해 혼합하여 쓸 건지' 결정해야 한다.

## train/test dataset 설명

1년 6개월치의 월별 고객데이터. 첫 1년 5개월은 훈련 데이터.
1. age, antiguedad, indrel_1mes 등의 수치 변수가 object. 정제작업이 필요하다
2. 대부분의 고객변수에 결측치가 있다. 수치형 / 범주형 변수의 결측치는 기존 변수에 없는 값 (0 or -1)으로 흔히 대체한다. 날짜 변수는 어느 날짜로 대체할지 고민이 필요함
3. 두 개의 고유값을 갖는 이진변수가 많다. 메모리 효율 극대화를 위해 int64 0, 1로 변환해야.
4. 고객 등급, 고객 관계유형 등 변수의 각 값이 무엇을 의미하는지 구체적인 설명이 부족하다. 
5. 예측하려는 게 '신규 구매'이므로, 제공된 데이터에서 '신규 구매'여부를 별도로 추출해야 한다. 평가 기준도 '신규 구매' 기준으로 진행되어야 한다.
6. 신규 구매 데이터는 계절성을 띄고 있다. 단일 모델을 쓸지, 계절별로 다른 모델을 쓸지 결정해야 한다. 다수의 모델을 서로 다른 계절 기반으로 학습하는 것도 방법이다.



# Baseline 모델

## 전처리

1. 제품 변수의 결측값을 0으로 대체한다. 제품 보유 여부 정보가 없으면 보유하고 있지 않다고 가정한다.
2. 훈련 / 테스트 데이터를 통합한다. 테스트 데이터에 없는 24개 제품변수는 0으로 채운다.
3. 범주형 / 수치형 데이터 전처리. 범주형은 .factorize()로 라벨 인코딩. object 타입일 경우 unique()로 특이값을 대체하거나 제거하고 정수형 데이터로 변환한다
4. 모델 학습에 사용할 변수는 features 항목에 미리 저장한다

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

# 데이터를 불러온다.
trn = pd.read_csv('../input/train_ver2.csv')
tst = pd.read_csv('../input/test_ver2.csv')

In [ ]:
## 데이터 전처리 ##

# 제품 변수를 별도로 저장해 놓는다.
prods = trn.columns[24:].tolist()

# 제품 변수 결측값을 미리 0으로 대체한다.
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

# 24개 제품 중 하나도 보유하지 않는 고객 데이터를 제거한다.
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

# 훈련 데이터와 테스트 데이터를 통합한다. 테스트 데이터에 없는 제품 변수는 0으로 채운다.
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

# 학습에 사용할 변수를 담는 list이다.
features = []

# 범주형 변수를 .factorize() 함수를 통해 label encoding한다.
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고, 정수형으로 변환한다.
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

# 학습에 사용할 수치형 변수를 features에 추가한다.
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

# (피쳐 엔지니어링) 두 날짜 변수에서 연도와 월 정보를 추출한다.
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

# 그 외 변수의 결측값은 모두 -99로 대체한다.
df.fillna(-99, inplace=True)

# Feature Engineering

머신러닝 모델 학습에 사용할 파생변수를 생성한다. baseline 모델에서는 24개 고객 변수, 4개의 날짜 기반 파생변수, 24개의 lag_1 변수를 사용했다.

고객이 첫 계약을 맺은 날짜인 fecha_alta, 고객이 마지막으로 1등급이었던 날짜 ult_fec_cli_1t 변수에서 각각 연도와 월 정보를 추출한다. ex) 두 개의 날짜변수의 차이값을 파생변수로 생성하거나, 졸업식 또는 방학 같은 특별한 날짜까지의 거리를 수치형 변수로 생성할 수도.

결측값은 임시로 -99로 대체한다. sklearn에서는 결측값을 입력값으로 받지 않고 실행에러를 반환하지만, xgboost는 결측값도 정상적인 입력값으로 받는다. 데이터가 결측되었다는 것도 하나의 정보로 인식하고 모델학습에 활용하지만, 이번 장에서는 결측값을 -99로 설정한다.

시계열 데이터는 고객의 과거데이터를 기반으로 다양한 파생변수를 만들 수 있다. 예컨대 고객 나이가 최근 3개월간 변동이 있었는지(3개월 안에 생일을 맞이했는지)를 이진변수로 생성하거나, 한 달 전에 구매한 제품변수를 사용하거나, 최근 6개월 평균월급을 계산할 수 있다.

이 경진대회의 경우 n개월 전에 금융제품을 보유하고 있었는지를 나타내는 lag변수가 좋은 파생변수로 작용했다. 24개의 금융제품 변수를 1개월 전, 2개월 전, 3개월 전 보유여부를 현재 고객 데이터로 활용하는 것.

baseline 모델에서는 1개월 전 정보만을 사용했다. lag_5까지 직접 구현해서 성능개선을 확인하는 걸 추천한다.


In [ ]:
# (피쳐 엔지니어링) lag-1 데이터를 생성한다.
# 코드 2-12와 유사한 코드 흐름이다.

# 날짜를 숫자로 변환하는 함수이다. 2015-01-28은 1, 2016-06-28은 18로 변환된다
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. Lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다
del df, df_lag

# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

# lag-1 변수를 추가한다.
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

###
### Baseline 모델 이후, 다양한 피쳐 엔지니어링을 여기에 추가한다.
###


# 모델 학습

[교차 검증]

경진대회에서 좋은 성적을 거두기 위해 가장 중요한 것. 경진대회 진행 중에는 참가자가 하루 최대 5개의 예측 결과물을 제출할 수 있다.

이런 시계열 데이터는 validation용 데이터를 가장 마지막 시간데이터로 두는 게 일반적. 예컨대 150128 ~ 160628 훈련 데이터라면 validation 데이터는 160528 한 달치, 나머지를 train 데이터로 구분한다.

이 baseline 모델에서는 간소화를 위해 160128 ~ 160428 4개월치 데이터를 훈련 데이터로, 160528 데이터를 validation으로 사용한다.

In [ ]:

## 모델 학습
# 학습을 위하여 데이터를 훈련, 테스트용으로 분리한다.
# 학습에는 2016-01-28 ~ 2016-04-28 데이터만 사용하고, 검증에는 2016-05-28 데이터를 사용한다.
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

# 훈련 데이터에서 신규 구매 건수만 추출한다.
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

# 훈련, 검증 데이터로 분리한다. 
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]



# 모델

xgboost 모델 사용. 대부분의 케글 상위입상자가 사용하는 모델.

파라미터 설명
* max_depth: 트리모델의 최대 깊이를 말한다. 값이 높을수록 더 복잡한 트리모델을 생성, overfit의 원인이 될 수 있다
* eta: 딥러닝의 lr과 같은 개념. 값이 너무 높으면 학습이 잘 안 될 수 있으며, 값이 너무 낮으면 학습속도가 느리다.
* colsample_bytree: 트리 생성시 훈련 데이터에서 변수를 샘플링해 주는 비율. 모든 트리는 전체 변수의 일부만을 학습하여 서로의 약점을 보완해주는 것. 보통 .6 ~ .9 값을 사용한다.
* colsample_bylevel: 트리의 레벨별로 훈련 데이터의 변수를 샘플링해주는 비율이다. .6 ~ .9 사이값 사용

모델의 최적 파라미터를 찾는 것도 중요하지만, 시간 대비 투자효율을 생각한다면 feature engineering에 더 많은 시간을 할애하는 걸 권장한다.

In [ ]:

# XGBoost 모델 parameter를 설정한다.
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

# 훈련, 검증 데이터를 XGBoost 형태로 변환한다.
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

# XGBoost 모델을 훈련 데이터로 학습한다!
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

# 학습한 모델을 저장한다.
import pickle
pickle.dump(model, open("../model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit


# Cross Validation

경진대회의 평가 척도를 사용하는 게 매우 중요하다. 이게 개선되어야 유의미하니까.

map@7 척도는 최고 점수가 데이터에 따라 변동할 수 있다. baseline 모델의 경우 검증데이터에서 얻을 수 있는 최고점수는 .04점대. (실제 정답값을 넣었을 때의 결과라고 함)

점수가 1보다 낮은 이유는, 검증 데이터의 모든 고객이 신규 구매를 하지 않았기 때문. 예컨대 100명 중 10명이 신규 구매를 했다면, 10명을 정확히 예측해도 10%의 map@7점수를 받게 된다.

In [ ]:

# MAP@7 평가 척도를 위한 준비작업이다.
# 고객 식별 번호를 추출한다.
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])
# 검증 데이터에서 신규 구매를 구한다.
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]    
add_vld = vld.as_matrix(columns=[prod + '_add' for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

# 고객별 신규 구매 정답 값을 add_vld_list에 저장하고, 총 count를 count_vld에 저장한다.
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

# 검증 데이터에서 얻을 수 있는 MAP@7 최고점을 미리 구한다. (0.042663)
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

# 검증 데이터에 대한 예측 값을 구한다.
X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

# 저번 달에 보유한 제품은 신규 구매가 불가하기 때문에, 확률값에서 미리 1을 빼준다
preds_vld = preds_vld - vld.as_matrix(columns=[prod + '_prev' for prod in prods])

# 검증 데이터 예측 상위 7개를 추출한다.
result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    result_vld.append([ip for y,p,ip in y_prods])
    
# 검증 데이터에서의 MAP@7 점수를 구한다. (0.036466)
print(mapk(add_vld_list, result_vld, 7, 0.0))

Baseline 모델은 검증 데이터에서 map@7 .036 정도 나온다.

검증 데이터 최고점수가 .04인 걸 감안하면 정확도는 .036 / .004 = 약 85% 정도.

# 테스트 데이터 예측 및 케글 업로드

테스트 데이터에 더 좋은 성능을 내기 위해서는 훈련+검증 데이터 합친 전체 데이터에 xgboost 모델을 다시 학습한다. 파라미터는 '교차검증'을 통해 찾아낸 최적의 파라미터를 사용하되, xgboost 모델에 사용되는 트리의 개수는 늘어난 검증 데이터만큼 증가한다.

학습한 모델의 변수 중요도는 get_fscore()로 확인할 수 있다.

케글 제출을 위해 제출파일을 생성한다. 

In [ ]:
import numpy as np

def apk(actual, predicted, k=7, default=0.0):
    # MAP@7 이므로, 최대 7개만 사용한다
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        # 점수를 부여하는 조건은 다음과 같다 :
        # 예측값이 정답에 있고 (‘p in actual’)
        # 예측값이 중복이 아니면 (‘p not in predicted[:i]’) 
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # 정답값이 공백일 경우, 무조건 0.0점을 반환한다
    if not actual:
        return default

    # 정답의 개수(len(actual))로 average precision을 구한다
    return score / min(len(actual), k)

def mapk(actual, predicted, k=7, default=0.0):
    # list of list인 정답값(actual)과 예측값(predicted)에서 고객별 Average Precision을 구하고, np.mean()을 통해 평균을 계산한다
    return np.mean([apk(a, p, k, default) for a, p in zip(actual, predicted)]) 


In [ ]:
# XGBoost 모델을 전체 훈련 데이터로 재학습한다!
X_all = XY.as_matrix(columns=features)
Y_all = XY.as_matrix(columns=['y'])
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]
# 트리 개수를 늘어난 데이터 양만큼 비례해서 증가한다.
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
# XGBoost 모델 재학습!
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

# 변수 중요도를 출력해본다. 예상하던 변수가 상위로 올라와 있는가?
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

# 캐글 제출을 위하여 테스트 데이터에 대한 예측 값을 구한다.
X_tst = tst.as_matrix(columns=features)
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst.as_matrix(columns=['ncodpers'])
preds_tst = preds_tst - tst.as_matrix(columns=[prod + '_prev' for prod in prods])

# 제출 파일을 생성한다.
submit_file = open('../model/xgb.baseline.2015-06-28', 'w')
submit_file.write('ncodpers,added_products\n')
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))



케글에서는 public score = 테스트 데이터의 70% 에 대한 평가점수를, private score는 나머지 30%에 대한 평가점수를 의미한다. 비율은 경진대회마다 다름

경진대회가 현재 진행형인 경우는 참가자에게 public score만 공개한다. 이 값을 기준으로 자신의 머신러닝 파이프라인 성능을 확인할 수 있다. 끝나는 날에는 private score공개, 최종순위는 private score 기준으로 정해진다.

참고로 이 baseline 모델은 전체 참가팀 기준으로 1787팀 중 1023등 정도라고.

이제 이 모델의 성능을 개선하는 일만 남은 셈이다. 모델 튜닝보다는 feature engineering에 더 많은 시간을 투자하는 것을 권하며, 데이터를 심도있게 고민하고 다양한 아이디어를 구현, 실험하는 것이 중요하다.

# 8등을 기록한 팀의 코드 분석
저작권: yaxinus

[참고사항]

- 데이터 전처리: 큰 효과를 본 건 없지만, 고객 데이터에서 날짜 관련 결측값을 채우려 시도했으며, age / antiguedad 변수를 수정했다. lag_5 이상의 데이터를 만들기 위해 14년 데이터를 생성하려 했지만 의미 없었다
- feature Engineering: 평범한 엔지니어링. lag_5, lag데이터에 대한 기초 통계값(최소, 최대, 표준편차)이 주된 엔지니어링이었다. 5 이상의 lag데이터로 성능 개선을 보지 못했다.
- 기본 모델 및 모델 통합: 다양한 변수, 가중치와 모델 설정값을 기준으로 mlogloss 기준으로 학습된 lightgbm, xgboost 기본 모델을 다수 만들었다. 초기 예측값을 기준으로 확률값을 계산하는 알고리즘을 사용해, 초기 예측값 대비 피어슨 상관관계가 가장 낮은 모델 결과물을 통합했다. 단순히 다수의 모델 결과물을 통합하는 것보다 이 편이 조금 더 좋은 성능개선을 보였다.



In [ ]:
# 훈련 데이터와 테스트 데이터를 하나의 데이터로 통합하는 코드이다.
def clean_data(fi, fo, header, suffix):
    
    # fi : 훈련/테스트 데이터를 읽어오는 file iterator
    # fo : 통합되는 데이터가 write되는 경로
    # header : 데이터에 header 줄을 추가할 것인지를 결정하는 boolean
    # suffix : 훈련 데이터에는 48개의 변수가 있고, 테스트 데이터에는 24개의 변수만 있다. suffix로 부족한 테스트 데이터 24개분을 공백으로 채운다.

    # csv의 첫줄, 즉 header를 읽어온다
    head = fi.readline().strip("\n").split(",")
    head = [h.strip('"') for h in head]

    # ‘nomprov’ 변수의 위치를 ip에 저장한다
    for i, h in enumerate(head):
        if h == "nomprov":
            ip = i

    # header가 True 일 경우에는, 저장할 파일의 header를 write한다
    if header:
        fo.write("%s\n" % ",".join(head))

    # n은 읽어온 변수의 개수를 의미한다 (훈련 데이터 : 48, 테스트 데이터 : 24)
    n = len(head)
    for line in fi:
        # 파일의 내용을 한줄 씩 읽어와서, 줄바꿈(\n)과 ‘,’으로 분리한다
        fields = line.strip("\n").split(",")

        # ‘nomprov’변수에 ‘,’을 포함하는 데이터가 존재한다. ‘,’으로 분리된 데이터를 다시 조합한다
        if len(fields) > n:
            prov = fields[ip] + fields[ip+1]
            del fields[ip]
            fields[ip] = prov

        # 데이터 개수가 n개와 동일한지 확인하고, 파일에 write한다. 테스트 데이터의 경우, suffix는24개의 공백이다
        assert len(fields) == n
        fields = [field.strip() for field in fields]
        fo.write("%s%s\n" % (",".join(fields), suffix))

# 하나의 데이터로 통합하는 코드를 실행한다. 먼저 훈련 데이터를 write하고, 그 다음으로 테스트 데이터를 write한다. 이제부터 하나의 dataframe만을 다루며 데이터 전처리를 진행한다.
with open("../input/8th.clean.all.csv", "w") as f:
    clean_data(open("../input/train_ver2.csv"), f, True, "")
    comma24 = "".join(["," for i in range(24)])
    clean_data(open("../input/test_ver2.csv"), f, False, comma24)


8등팀의 main.py 파일에서 
1. 데이터 전처리
2. feature engineering
3. 모델 학습
4. test data예측 / 케글 업로드

일련의 머신러닝 파이프라인 과정을 모두 수행했다.

main.py가 메인이며, 관련 라이브러리와 머신러닝 함수가 포함된 게 engine.py, 자주 사용되는 함수가 포함된 utils.py로 함께 import한다.

main.py에서 데이터 전처리와 feature engineering은 make_data() 함수가, 모델 학습 및 케글 제출용 파일은 train_predict()에서 담당한다.

# 데이터 전처리 + feature engineering

make_data() 함수에서 동시에 처리한다,. 48개 변수 각각에 맞춤화된 전처리 / 엔지니어링 제공.


In [ ]:
import math    
import io    

# 파일 압축 용도
import gzip    
import pickle    
import zlib    

# 데이터, 배열을 다루기 위한 기본 라이브러리
import pandas as pd 
import numpy as np

# 범주형 데이터를 수치형으로 변환하기 위한 전처리 도구
from sklearn.preprocessing import LabelEncoder

import engines
from utils import *
np.random.seed(2016)
transformers = {}

In [ ]:
def load_data():
    # “데이터 준비”에서 통합한 데이터를 읽어온다
    fname = "../input/8th.clean.all.csv"
    train_df = pd.read_csv(fname, dtype=dtypes)

    # products는 util.py에서 정의한 24개의 금융 제품이름이다
    # 결측값을 0.0으로 대체하고, 정수형으로 변환한다
    for prod in products:
        train_df[prod] = train_df[prod].fillna(0.0).astype(np.int8)

    # 48개의 변수마다 전처리/피처 엔지니어링을 적용한다
    train_df, features = apply_transforms(train_df)

    prev_dfs = []
    prod_features = None

    use_features = frozenset([1,2])
    # 1 ~ 5까지의 step에 대하여 make_prev_df()를 통해 lag-n 데이터를 생성한다
    for step in range(1,6):
        prev1_train_df, prod1_features = make_prev_df(train_df, step)
        # 생성한 lag 데이터는 prev_dfs 리스트에 저장한다
        prev_dfs.append(prev1_train_df)
        # features에는 lag-1,2만 추가한다
        if step in use_features:
            features += prod1_features
        # prod_features에는 lag-1의 변수명만 저장한다
        if step == 1:
            prod_features = prod1_features

    return train_df, prev_dfs, features, prod_features


24개 금융변수 결측치 대체, 정수형으로 변환한 후 apply_function() 사용해 데이터 전처리 / feature engineering 수행.

apply_funcion() 설명하기 전, 이 함수에 포함된 4개 도구를 먼저 요약하면

1. label_encoder(df, features, name) = df에서 범주형 변수 name을 LabelEncoder()를 써서 수치형으로 변환한다. 사전에 정의한 dict()인 transformers에 label encoding 수행한 변수명을 기록함


== df에 동일한 변수를 label encoding할 때는 기존의 LabelEncoder()를 재활용한다. 단 실제 코드에서는 재활용된 적이 한 번도 없다.


In [ ]:
def label_encode(df, features, name):
    # 데이터 프레임 df의 변수 name의 값을 모두 string으로 변환한다
    df[name] = df[name].astype('str')
    # 이미, label_encode 했던 변수일 경우, transformer[name]에 있는 LabelEncoder()를 재활용한다
    if name in transformers:
        df[name] = transformers[name].transform(df[name])
    # 처음 보는 변수일 경우, transformer에 LabelEncoder()를 저장하고, .fit_transform() 함수로 label encoding을 수행한다
    else: # train
        transformers[name] = LabelEncoder()
        df[name] = transformers[name].fit_transform(df[name])
    # label encoding한 변수는 features 리스트에 추가한다
    features.append(name)

2. encode_top(s, count=100, dtype=np.int8) 함수는 Series에서 빈도가 높은 100개 고유값을 순위로 대체하고, 그 외 빈도가 낮은 값을 0으로 변환한 새로운 Series를 반환한다. 데이터 전체가 아닌, 고빈도 데이터 정보를 추출하기 위해 사용함.

In [ ]:
def encode_top(s, count=100, dtype=np.int8):
    # 모든 고유값에 대한 빈도를 계산한다
    uniqs, freqs = np.unique(s, return_counts=True)
    # 빈도 Top 100을 추출한다
    top = sorted(zip(uniqs,freqs), key=lambda vk: vk[1], reverse = True)[:count]
    # { 기존 데이터 : 순위 } 를 나타내는 dict()를 생성한다
    top_map = {uf[0]: l+1 for uf, l in zip(top, range(len(top)))}
    # 고빈도 100개의 데이터는 순위로 대체하고, 그 외는 0으로 대체한다
    return s.map(lambda x: top_map.get(x, 0)).astype(dtype)

3. data_to_float(str_date) 함수는 입력으로 들어오는 날짜 데이터를 숫자로 변환하는 함수. 입력값이 결측치인 경우 np.nan을 반환하지만, 입력값이 문자열인 형태의 날짜 데이터일 경우 (연도 * 12 + 월) 계산으로 날짜 데이터를 소수로 환산해 반환한다. (날짜 데이터를 단순 월 단위 수치형 데이터로 변경한 것). date_to_int(str_date)는 월 단위 수치형으로 변환된 데이터를 1~18 사이의 값으로 변환한다.

utils.py에 있다.

In [ ]:
def date_to_float(str_date):
    if str_date.__class__ is float and math.isnan(str_date) or str_date == "":
        return np.nan
    Y, M, D = [int(a) for a in str_date.strip().split("-")]
    float_date = float(Y) * 12 + float(M)
    return float_date

# 날짜 데이터를 월 단위 숫자로 변환하되 1~18 사이로 제한하는 함수
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] # "2016-05-28"
    int_date = (int(Y) - 2015) * 12 + int(M)
    assert 1 <= int_date <= 12 + 6
    return int_date

4. custom_one_hot(df, features, name, names) 함수는 범주형 변수를 입력받아, 변수 안에 존재하는 고유값을 새로운 이진 변수로 생성하는 one hot encoding을 수행한다.

**범주형 데이터를 하나의 열에서 label encoding하는 것보다 표현력이 높아지지만 고유값의 숫자만큼 데이터의 열이 늘어나기 때문에, 고유값이 적은 데이터에서 선호되는 엔지니어링 기법**

sklearn의 onehotencoding이나 pd.get_dummies 로도 만들 수 있으나 여기서는 직접 구현했다고 함


In [ ]:
def custom_one_hot(df, features, name, names, dtype=np.int8, check=False):
    for n, val in names.items():
        # 신규 변수명을 “변수명_숫자”로 지정한다
        new_name = "%s_%s" % (name, n)
        # 기존 변수에서 해당 고유값을 가지면 1, 그 외는 0인 이진 변수를 생성한다
        df[new_name] = df[name].map(lambda x: 1 if x == val else 0).astype(dtype)
        features.append(new_name)

### apply_transform

4개의 도구함수를 이용해 총 48개의 변수에 대한 데이터 전처리와 엔지니어링 수행. 주된 전처리 내용과 엔지니어링 내용은

- 결측값 대체: 데이터 내에 존재하는 결측치를 0 똔느 1로 대체한다
- 범주형 데이터 label encoding: 범주형으로 표현되는 데이터를 sklearn.preprocessing의 LabelEncoder 도구를 써서 수치형으로 변환했다
- 고빈도 top 100를 빈도 순위로 변환: 특정 변수에서 빈도가 높은 값을 순위로 변환하여 고빈도 데이터에 대한 선형관계를 추출했다
- 수치형 변수 Log transformation: log transformationdms 데이터 내 대소관계를 유지하면서 포함된 값들의 차이를 줄여주는 역할
- 날짜 데이터에서 '연,월' 추출: 문자열 데이터에서 연도와 월 추출
- 날짜 데이터 간의 차이값으로 파생변수 생성: 2개의 날짜 데이터 차이값을 통해 상대적인 거리 변수를 생성한다.
- one hot encoding 변수 생성: 범주형 데이터의 표현력을 높이기 위해, 모든 고유값을 새로운 이진 변수로 생성한다.


In [ ]:
def apply_transforms(train_df):
    # 학습에 사용할 변수를 저장할 features 리스트를 생성한다
    features = []

    # 두 변수를 label_encode() 한다
    label_encode(train_df, features, "canal_entrada")
    label_encode(train_df, features, "pais_residencia")

    # age의 결측값을 0.0으로 대체하고, 모든 값을 정수로 변환한다.
    train_df["age"] = train_df["age"].fillna(0.0).astype(np.int16)
    features.append("age")

    # renta의 결측값을 1.0으로 대체하고, log를 씌워 분포를 변형한다
    train_df["renta"].fillna(1.0, inplace=True)
    train_df["renta"] = train_df["renta"].map(math.log)
    features.append("renta")

    # 고빈도 100개의 순위를 추출한다
    train_df["renta_top"] = encode_top(train_df["renta"])
    features.append("renta_top")

    # 결측값 혹은 음수를 0으로 대체하고, 나머지 값은 +1.0 은 한 후에, 정수로 변환한다
    train_df["antiguedad"] = train_df["antiguedad"].map(lambda x: 0.0 if x < 0 or math.isnan(x) else x+1.0).astype(np.int16)
    features.append("antiguedad")

    # 결측값을 0.0으로 대체하고, 정수로 변환한다
    train_df["tipodom"] = train_df["tipodom"].fillna(0.0).astype(np.int8)
    features.append("tipodom")
    train_df["cod_prov"] = train_df["cod_prov"].fillna(0.0).astype(np.int8)
    features.append("cod_prov")

    # fecha_dato에서 월/년도를 추출하여 정수값으로 변환한다
    train_df["fecha_dato_month"] = train_df["fecha_dato"].map(lambda x: int(x.split("-")[1])).astype(np.int8)
    features.append("fecha_dato_month")
    train_df["fecha_dato_year"] = train_df["fecha_dato"].map(lambda x: float(x.split("-")[0])).astype(np.int16)
    features.append("fecha_dato_year")

    # 결측값을 0.0으로 대체하고, fecha_alta에서 월/년도를 추출하여 정수값으로 변환한다
    # x.__class__는 결측값일 경우 float를 반환하기 때문에, 결측값 탐지용으로 사용하고 있다
    train_df["fecha_alta_month"] = train_df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[1])).astype(np.int8)
    features.append("fecha_alta_month")
    train_df["fecha_alta_year"] = train_df["fecha_alta"].map(lambda x: 0.0 if x.__class__ is float else float(x.split("-")[0])).astype(np.int16)
    features.append("fecha_alta_year")

    # 날짜 데이터를 월 기준 수치형 변수로 변환한다
    train_df["fecha_dato_float"] = train_df["fecha_dato"].map(date_to_float)
    train_df["fecha_alta_float"] = train_df["fecha_alta"].map(date_to_float)

    # fecha_dato 와 fecha_alto의 월 기준 수치형 변수의 차이값을 파생 변수로 생성한다
    train_df["dato_minus_alta"] = train_df["fecha_dato_float"] - train_df["fecha_alta_float"]
    features.append("dato_minus_alta")

    # 날짜 데이터를 월 기준 수치형 변수로 변환한다 (1 ~ 18 사이 값으로 제한)
    train_df["int_date"] = train_df["fecha_dato"].map(date_to_int).astype(np.int8)

    # 자체 개발한 one-hot-encoding을 수행한다
    custom_one_hot(train_df, features, "indresi", {"n":"N"})
    custom_one_hot(train_df, features, "indext", {"s":"S"})
    custom_one_hot(train_df, features, "conyuemp", {"n":"N"})
    custom_one_hot(train_df, features, "sexo", {"h":"H", "v":"V"})
    custom_one_hot(train_df, features, "ind_empleado", {"a":"A", "b":"B", "f":"F", "n":"N"})
    custom_one_hot(train_df, features, "ind_nuevo", {"new":1})
    custom_one_hot(train_df, features, "segmento", {"top":"01 - TOP", "particulares":"02 - PARTICULARES", "universitario":"03 - UNIVERSITARIO"})
    custom_one_hot(train_df, features, "indfall", {"s":"S"})
    custom_one_hot(train_df, features, "tiprel_1mes", {"a":"A", "i":"I", "p":"P", "r":"R"}, check=True)
    custom_one_hot(train_df, features, "indrel", {"1":1, "99":99})

    # 결측값을 0.0으로 대체하고, 그 외는 +1.0을 더하고, 정수로 변환한다
    train_df["ind_actividad_cliente"] = train_df["ind_actividad_cliente"].map(lambda x: 0.0 if math.isnan(x) else x+1.0).astype(np.int8)
    features.append("ind_actividad_cliente")

    # 결측값을 0.0으로 대체하고, “P”를 5로 대체하고, 정수로 변환한다
    train_df["indrel_1mes"] = train_df["indrel_1mes"].map(lambda x: 5.0 if x == "P" else x).astype(float).fillna(0.0).astype(np.int8)
    features.append("indrel_1mes")
    
    # 데이터 전처리/피쳐 엔지니어링이 1차적으로 완료된 데이터 프레임 train_df와 학습에 사용할 변수 리스트 features를 tuple 형태로 반환한다
    return train_df, tuple(features)


# 금융 변수의 lag 데이터 생성

시계열 문제에서 많이 사용되는 파생변수의 하나, 해당 변수의 n단위 이전 값을 lag_n 데이터라고 칭한다. 시계열 경진대회에서는 lag_n 데이터가 유의미한 성능개선을 보이는 경우가 종종 있다.

2개의 도구함수
1. make_prev_df(train_df, step) = 24개 금융변수의 Lag 데이터를 직접 생성. 날짜 데이터를 1~18 사이의 정수로 반환한 int_date 변수를 써서 24개 금융변수 값을 step 개월 이동시켜 lag 변수를 만든다

In [ ]:
def make_prev_df(train_df, step):
    # 새로운 데이터 프레임에 ncodpers를 추가하고, int_date를 step만큼 이동시킨 값을 넣는다
    prev_df = pd.DataFrame()
    prev_df["ncodpers"] = train_df["ncodpers"]
    prev_df["int_date"] = train_df["int_date"].map(lambda x: x+step).astype(np.int8)

    # “변수명_prev1” 형태의 lag 변수를 생성한다
    prod_features = ["%s_prev%s" % (prod, step) for prod in products]
    for prod, prev in zip(products, prod_features):
        prev_df[prev] = train_df[prod]

    return prev_df, tuple(prod_features)

2. join_with_prev(df, prev_df, how) 함수는 기존 train_df에 lag 데이터 조인하는 함수

In [ ]:
def join_with_prev(df, prev_df, how):
    # pandas merge 함수를 통해 join
    df = df.merge(prev_df, on=["ncodpers", "int_date"], how=how)
    # 24개 금융 변수를 소수형으로 변환한다
    for f in set(prev_df.columns.values.tolist()) - set(["ncodpers", "int_date"]):
        df[f] = df[f].astype(np.float16)
    return df

최대 lag_5까지의 변수를 생성하고 train_df에 조인한다. 이 경진대회에서 가장 중요한 변수가 Lag_5였던 만큼, 이 과정이 있고 없고가 성능에 큰 영향을 준다

In [ ]:
def load_data():
    # “데이터 준비”에서 통합한 데이터를 읽어온다
    fname = "../input/8th.clean.all.csv"
    train_df = pd.read_csv(fname, dtype=dtypes)

    # products는 util.py에서 정의한 24개의 금융 제품이름이다
    # 결측값을 0.0으로 대체하고, 정수형으로 변환한다
    for prod in products:
        train_df[prod] = train_df[prod].fillna(0.0).astype(np.int8)

    # 48개의 변수마다 전처리/피처 엔지니어링을 적용한다
    train_df, features = apply_transforms(train_df)

    prev_dfs = []
    prod_features = None

    use_features = frozenset([1,2])
    # 1 ~ 5까지의 step에 대하여 make_prev_df()를 통해 lag-n 데이터를 생성한다
    for step in range(1,6):
        prev1_train_df, prod1_features = make_prev_df(train_df, step)
        # 생성한 lag 데이터는 prev_dfs 리스트에 저장한다
        prev_dfs.append(prev1_train_df)
        # features에는 lag-1,2만 추가한다
        if step in use_features:
            features += prod1_features
        # prod_features에는 lag-1의 변수명만 저장한다
        if step == 1:
            prod_features = prod1_features

    return train_df, prev_dfs, features, prod_features

# 특이점은, lag_5 변수를 생성하면서도 features에는 lag 1,2내용만 추가하고, prod_features에는 lag_1 변수명만 저장했다

make_data()의

``    for i, prev_df in enumerate(prev_dfs):
        with Timer("join train with prev%s" % (i+1)):
            how = "inner" if i == 0 else "left"
            train_df = join_with_prev(train_df, prev_df, how=how)``
            
   여기가 lag데이터 통합하는 부분. lag 1만 inner join, 나머지는 left join
  

또한 lag 구간별로 평균 / 표준편차 / 최댓값 / 최솟값을 구해 데이터에 추가한다.

```
# 24개의 금융 변수에 대해서 for loop을 돈다
    for prod in products:
        # [1~3], [1~5], [2~5] 의 3개 구간에 대해서 표준편차를 구한다
        for begin, end in [(1,3),(1,5),(2,5)]:
            prods = ["%s_prev%s" % (prod, i) for i in range(begin,end+1)]
            mp_df = train_df.as_matrix(columns=prods)
            stdf = "%s_std_%s_%s" % (prod,begin,end)

            # np.nanstd로 표준편차를 구하고, features에 신규 파생 변수 이름을 추가한다
            train_df[stdf] = np.nanstd(mp_df, axis=1)
            features += (stdf,)

        # [2~3], [2~5] 의 2개 구간에 대해서 최소값/최대값을 구한다
        for begin, end in [(2,3),(2,5)]:
            prods = ["%s_prev%s" % (prod, i) for i in range(begin,end+1)]
            mp_df = train_df.as_matrix(columns=prods)

            minf = "%s_min_%s_%s"%(prod,begin,end)
            train_df[minf] = np.nanmin(mp_df, axis=1).astype(np.int8)

            maxf = "%s_max_%s_%s"%(prod,begin,end)
            train_df[maxf] = np.nanmax(mp_df, axis=1).astype(np.int8)

            features += (minf,maxf,)
```

lag의 기초통계 변수를 명시적으로 변수화해서, 학습 모델이 lag 변수에 숨겨진 패턴을 더 찾기 쉽게 하도록 돕는 셈.

그리고 마지막으로 train_df에서 학습에 필요한 변수만을 빼내는 코드가

``` 
# 고객 고유 식별 번호(ncodpers), 정수로 표현한 날짜(int_date), 실제 날짜(fecha_dato), 24개의 금융 변수(products)와 학습에 사용하기 위해 전처리/피쳐 엔지니어링한 변수(features)가 주요 변수이다.
    leave_columns = ["ncodpers", "int_date", "fecha_dato"] + list(products) + list(features)
    # 중복값이 없는지 확인한다
    assert len(leave_columns) == len(set(leave_columns))
    # train_df에서 주요 변수만을 추출한다
    train_df = train_df[leave_columns]

    return train_df, features, prod_features
    ```
   
   
   가 된다.
   
 
 
 

features는 학습에 사용하기 위한 변수를 기록한 튜플값. 결측값 대체, log transform, label encoding, one hot encoding, lag 데이터 등 변수명이 포함되어 있다

prdo_features는 lag_1 데이터 변수명을 저장한 튜플

```
if __name__ == "__main__":
    all_df, features, prod_features = make_data()
    
    # 피쳐 엔지니어링이 완료된 데이터를 저장한다
    train_df.to_pickle("../input/8th.feature_engineer.all.pkl")
    pickle.dump((features, prod_features), open("../input/8th.feature_engineer.cv_meta.pkl", "wb"))

    train_predict(all_df, features, prod_features, "2016-05-28", cv=True)
    train_predict(all_df, features, prod_features, "2016-06-28", cv=False)
``` 

main 함수 마지막 부분. 데이터를 압축파일 형태로 저장한다.

# 모델 학습

train_predict() 함수는 머신러닝 모델학습을 위한 준비과정을 마친 후, lightgbm과 xgboost 모델을 학습한다.

첫 번째 train_predict() 함수는 훈련 데이터의 최신 날짜인 160528을 테스트 데이터와 같이 사용, 그 외 데이터는 훈련과 검증 데이터 8:2 비율로 분리한다. 

- 검증 데이터를 기반으로 모델의 최적 파라미터를 결정하고,
- 모든 훈련 데이터를 머신러닝 모델에 다시 학습시킨 다음,
- 테스트 데이터에 대한 예측값을 만들어낸다.
이 때 테스트 데이터의 정답값을 기반으로 경진대회 평가척도 값을 확인해서, 케글에 업로드하기 전 모델의 성능을 자체적으로 검증한다.

훈련 데이터 안에서 훈련/검증/테스트를 분리하는 이뉴는 최대한 객관적인 모델 성능을 얻기 위해서.

두 번째는 실제 훈련데이터 전부 써서, 케글에 제출할 예측 결과물을 만들어낸다.


train_predict()는 머신러닝 학습 직전에 준비과정을 수행한다. 제품 보유여부를 의미하는 금융변수에서 신규 구매정보를 추출함.

탐색적 데이터 분석, baseline 모델에서도 지적됐듯, 제품보유와 신규구매의 차이를 인지하고 신규구매 정보를 추출하는 작업을 수행한다. 신규 구매가 존재하는 데이터를 유효한 데이터로 인식.

이 과정에서 1명 고객이 같은 날 2개 이상의 금융제품을 신규구매할 경우... '구매활동량'이 높은 고객 데이터가 필요 이상으로 많아져, 신규구매가 적거나 평범한 고객의 정확도가 낮아질 수 있다. xgboost와 ligthgbm은 이런 데이터 분포를 고려해 '각 고객 데이터의 가중치를 부여하는 weight변수를 지원한다'. 신규 구매건수가 많은 고객의 경우 weight을 낮게 배정하여 올바른 데이터분포를 유지하는 것.

In [ ]:
def train_predict(all_df, features, prod_features, str_date, cv):
    # all_df : 통합 데이터
    # features : 학습에 사용할 변수
    # prod_features : 24개 금융 변수
    # str_date : 예측 결과물을 산출하는 날짜. 2016-05-28일 경우, 훈련 데이터의 일부이며 정답을 알고 있기에 교차 검증을 의미하고, 2016-06-28일 경우, 캐글에 업로드하기 위한 테스트 데이터 예측 결과물을 생성한다
    # cv : 교차 검증 실행 여부

    # str_date로 예측 결과물을 산출하는 날짜를 지정한다
    test_date = date_to_int(str_date)
    # 훈련 데이터는 test_date 이전의 모든 데이터를 사용한다
    train_df = all_df[all_df.int_date < test_date]
    # 테스트 데이터를 통합 데이터에서 분리한다
    test_df = pd.DataFrame(all_df[all_df.int_date == test_date])

    # 신규 구매 고객만을 훈련 데이터로 추출한다
    X = []
    Y = []
    for i, prod in enumerate(products):
        prev = prod + "_prev1"
        # 신규 구매 고객을 prX에 저장한다
        prX = train_df[(train_df[prod] == 1) & (train_df[prev] == 0)]
        # prY에는 신규 구매에 대한 label 값을 저장한다
        prY = np.zeros(prX.shape[0], dtype=np.int8) + i
        X.append(prX)
        Y.append(prY)

    XY = pd.concat(X)
    Y = np.hstack(Y)
    # XY는 신규 구매 데이터만 포함한다
    XY["y"] = Y

    # 메모리에서 변수 삭제
    del train_df
    del all_df

    # 데이터별 가중치를 계산하기 위해서 새로운 변수 (ncodpers + fecha_dato)를 생성한다
    XY["ncodepers_fecha_dato"] = XY["ncodpers"].astype(str) + XY["fecha_dato"]
    uniqs, counts = np.unique(XY["ncodepers_fecha_dato"], return_counts=True)
    # 자연 상수(e)를 통해서, count가 높은 데이터에 낮은 가중치를 준다
    weights = np.exp(1/counts - 1)

    # 가중치를 XY 데이터에 추가한다
    wdf = pd.DataFrame()
    wdf["ncodepers_fecha_dato"] = uniqs
    wdf["counts"] = counts
    wdf["weight"] = weights
    XY = XY.merge(wdf, on="ncodepers_fecha_dato")
####################################################################################
    # 교차 검증을 위하여 XY를 훈련:검증 (8:2)로 분리한다
    mask = np.random.rand(len(XY)) < 0.8
    XY_train = XY[mask]
    XY_validate = XY[~mask]

    # 테스트 데이터에서 가중치는 모두 1이다
    test_df["weight"] = np.ones(len(test_df), dtype=np.int8)

    # 테스트 데이터에서 “신규 구매” 정답값을 추출한다. 
    test_df["y"] = test_df["ncodpers"]
    Y_prev = test_df.as_matrix(columns=prod_features)
    for prod in products:
        prev = prod + "_prev1"
        padd = prod + "_add"
        # 신규 구매 여부를 구한다
        test_df[padd] = test_df[prod] - test_df[prev]

    test_add_mat = test_df.as_matrix(columns=[prod + "_add" for prod in products])
    C = test_df.as_matrix(columns=["ncodpers"])
    test_add_list = [list() for i in range(len(C))]
    # 평가 척도 MAP@7 계산을 위하여, 고객별 신규 구매 정답값을 test_add_list에 기록한다
    count = 0
    for c in range(len(C)):
        for p in range(len(products)):
            if test_add_mat[c,p] > 0:
                test_add_list[c].append(p)
                count += 1
    
    # 교차 검증에서, 테스트 데이터로 분리된 데이터가 얻을 수 있는 최대 MAP@7 값을 계산한다. 
    if cv:
        max_map7 = mapk(test_add_list, test_add_list, 7, 0.0)
        map7coef = float(len(test_add_list)) / float(sum([int(bool(a)) for a in test_add_list]))
        print("Max MAP@7", str_date, max_map7, max_map7*map7coef)

    # LightGBM 모델 학습 후, 예측 결과물을 저장한다
    Y_test_lgbm = engines.lightgbm(XY_train, XY_validate, test_df, features, XY_all = XY, restore = (str_date == "2016-06-28"))
    test_add_list_lightgbm = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.lightgbm.csv.gz" % str_date, "wb"), Y_test_lgbm - Y_prev, C)

    # 교차 검증일 경우, LightGBM 모델의 테스트 데이터 MAP@7 평가 척도를 출력한다
    if cv:
        map7lightgbm = mapk(test_add_list, test_add_list_lightgbm, 7, 0.0)
        print("LightGBMlib MAP@7", str_date, map7lightgbm, map7lightgbm*map7coef)

    # XGBoost 모델 학습 후, 예측 결과물을 저장한다
    Y_test_xgb = engines.xgboost(XY_train, XY_validate, test_df, features, XY_all = XY, restore = (str_date == "2016-06-28"))
    test_add_list_xgboost = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.xgboost.csv.gz" % str_date, "wb"), Y_test_xgb - Y_prev, C)

    # 교차 검증일 경우, XGBoost 모델의 테스트 데이터 MAP@7 평가 척도를 출력한다
    if cv:
        map7xgboost = mapk(test_add_list, test_add_list_xgboost, 7, 0.0)
        print("XGBoost MAP@7", str_date, map7xgboost, map7xgboost*map7coef)

    # 곱셈 후, 제곱근을 구하는 방식으로 앙상블을 수행한다
    Y_test = np.sqrt(np.multiply(Y_test_xgb, Y_test_lgbm))
    # 앙상블 결과물을 저장하고, 테스트 데이터에 대한 MAP@7 를 출력한다
    test_add_list_xl = make_submission(io.BytesIO() if cv else gzip.open("tmp/%s.xgboost-lightgbm.csv.gz" % str_date, "wb"), Y_test - Y_prev, C)

    # 정답값인 test_add_list와 앙상블 모델의 예측값을 mapk 함수에 넣어, 평가 척도 점수를 확인한다
    if cv:
        map7xl = mapk(test_add_list, test_add_list_xl, 7, 0.0)
        print("XGBoost+LightGBM MAP@7", str_date, map7xl, map7xl*map7coef)


여기서는 lightgbm, xgboost 각각 하나의 모델을 학습하고, 각 모델의 결과물을 앙상블한 최종 결과물을 생성한다.

실제로는 수많은 모델을 학습해 그 결과물을 케글에 제출하였을 것이다.

다수의 모델 결과물을 생성하고 앙상블을 수행하는 건 경진대회에서 일반적인 전략이다. 가장 일반적인 앙상블은 각 모델의 결과물 산술평균을 계산하는 방법임. 단 이 코드에서는 기하 평균으로 앙상블을 계산함

### 모델 학습

모델 학습은 engines.py 파일이 담당한다. lightgbm, xgboost 두 함수의 입력값은 동일하며, 함수의 기본 흐름은

1. 교차 검증에 사용되는 훈련 데이터 XY_train, 검증 데이터 XY_validate를 기반으로 최적의 파라미터를 찾는다. 모델 학습 시에는 features 변수를 사용한다.
2. 전체 훈련 데이터(XY_all)에 최적의 파라미터를 기반으로 모델 학습을 한 후, test_df을 넣고 예측 결과물을 출력한다

### 예측 결과물은 make_submission() 함수로 케글 제출용 파일로 저장된다.


cf. baseline 모델의 xgboost 파라미터와 8등 팀의 xgboost모델 파라미터는 동일하다. feature engineering 차이가 낳은 결과임.

요약

1. 범주형 변수는 자체 구현한 label encoder onehotencoder로 수치형 변수로 변환
2. 빈도값 기준 상위 100개 데이터를 별도로 순위 표현하는 encoder_top()을 통해 파생변수 생성
3. 변수 범위가 큰 renta 변수는 log 취해서 정규화
4. 날짜 변수간의 차이값을 파생변수로 활용
5. lag_5 변수 활용 - 구간별 표준편차, 최솟값, 최댓값을 구해 다양한 엔지니어링을 구사함.
6. 같은 고객이 두 개 이상의 제품을 구매했을 때 생기는 빈도수 차이를 조절하기 위해 weight 값을 사용함

